In [1]:
from elasticsearch import Elasticsearch
import sqlite3
import json

## 1. Initialization

In [2]:
es = Elasticsearch(hosts='localhost')

In [3]:
mappings = {
        "mappings": {
            "properties": {
                "id": {
                    "type": 'integer'
                },
                "url": {
                    "type": "text"
                },
                "title": {
                    "type": "text",
                    "analyzer": "english",
                    "fielddata": True
                },
                "author": {
                    "type": "text",
                    "analyzer": "english",
                    "fielddata": True
                },
                "date": {
                    "type": "text",
                    "fielddata": True
                },
                "article": {
                    "type": "text",
                    "analyzer": "english"
                },
                "images": {
                    "type": "text",
                },
                "image_descriptions": {
                    "type": "text"
                }
            }
        }
    }
es.indices.create(index='msn', body=mappings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'msn'}

## 2. Load Data

In [4]:
db = sqlite3.connect('../data/MSN.db')
cursor = db.cursor()
sql_text = 'SELECT * FROM MSN'
info_all = list(cursor.execute(sql_text))

In [5]:
for i, (_id, url, title, author, date, article, images, _) in enumerate(info_all):
    print(i, title)
    images = eval(images)

    es.index(index='msn', body=json.dumps({
        'id': _id,
        'url': url,
        'title': title,
        'author': author,
        'date': date,
        'article': article,
        'images': str(images),
        "image_descriptions": str([image[1] for image in images])
    }))

g on how you downloaded them
15643 Florida counties use Eventbrite to schedule COVID-19 vaccine appointments
15644 Tesla appears to have plenty of momentum after meteoric 2020
15645 Cyberpunk 2077 Now Has A Third-Person Mod On PC
15646 Alien space junk may have passed through our solar system, professor says
15647 Windows 10 Update Will Get Rid of Flash Once and for All
15648 What Your Business Needs to Know About SERPs
15649 Boost your mood with emotional brain training
15650 Google may be updating the Nest Hub by adding Soli gestures
15651 OnePlus 9 Pro rumored to offer faster 45W wireless charging
15652 After almost four years, the last 'Nier: Automata' secret has been found
15653 Valorant Leak Reveals New Stealthy Agent And His Abilities
15654 Free Steam program lets you make your own Vtuber
15655 Why Slack Technologies Stock Soared 88% in 2020
15656 Byton partners with Foxconn, but extends shutdown
15657 xCloud now has 22 games that are optimized with touch controls
15658 Elon Mus

## 3. Cleanups

In [6]:
db.close()
es.close()